In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
from torchtext.vocab import build_vocab_from_iterator
from torch import nn
from tqdm.notebook import tqdm
import torch.nn as nn
from tensorflow import keras
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import TensorDataset, DataLoader
from nltk.stem import WordNetLemmatizer
from torchtext.data.functional import to_map_style_dataset
import nltk
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense,Input, Embedding,LSTM,Dropout,Conv1D, MaxPooling1D, GlobalMaxPooling1D,Dropout,Bidirectional,Flatten,BatchNormalization,SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
%matplotlib inline

In [4]:
data = pd.read_csv('/Users/paperplanes/Downloads/hw_machine_learning/train 2.csv')

In [62]:
data.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [20]:
train, test= train_test_split(data, test_size=0.2, random_state=42)
Xtrain, ytrain = train['full_text'], train['score']
Xtest, ytest = test['full_text'], test['score']

In [23]:
MAX_VOCAB_SIZE = 10000
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE,oov_token="<oov>")
tokenizer.fit_on_texts(Xtrain)
word_index = tokenizer.word_index
#print(word_index)
V = len(word_index)

In [24]:
seq_train = tokenizer.texts_to_sequences(Xtrain)
seq_test =  tokenizer.texts_to_sequences(Xtest)

In [14]:
seq_len_list = [len(i) for i in seq_train + seq_test]
max_len=max(seq_len_list)
max_seq_len = np.mean(seq_len_list) + 2 * np.std(seq_len_list)
max_seq_len = int(max_seq_len)

In [15]:
perc_covered = np.sum(np.array(seq_len_list) < max_seq_len) / len(seq_len_list)*100
pad_train=pad_sequences(seq_train,truncating = 'post', padding = 'pre',maxlen=max_seq_len)
pad_test=pad_sequences(seq_test,truncating = 'post', padding = 'pre',maxlen=max_seq_len)

In [57]:
Xtrain,Xval,ytrain,yval=train_test_split(pad_train,ytrain,
                                             test_size=0.2,random_state=10)

In [40]:
V = 50000

In [59]:
def lstm_model(Xtrain,Xval,ytrain,yval,V,D,maxlen,epochs):

    print("----Building the model----")
    i = Input(shape=(maxlen,))
    x = Embedding(V + 1, D,input_length = maxlen)(i)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Conv1D(32,5,activation = 'relu')(x)
    x = Dropout(0.3)(x)
    x = MaxPooling1D(2)(x)
    x = Bidirectional(LSTM(128,return_sequences=True))(x)
    x = LSTM(64)(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(i, x)
    model.summary()

    print("----Training the network----")
    model.compile(optimizer= Adam(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    r = model.fit(Xtrain,ytrain, 
                  validation_data = (Xval,yval), 
                  epochs = epochs, 
                  verbose = 2,
                  batch_size = 32)
                  #callbacks = callbacks
    print("Train score:", model.evaluate(Xtrain,ytrain))
    print("Validation score:", model.evaluate(Xval,yval))
    n_epochs = len(r.history['loss'])
    
    return r,model,n_epochs 

In [60]:
D =500
epochs = 5
r,model,n_epochs = lstm_model(Xtrain,Xval,ytrain,yval,V,D,max_seq_len,epochs)


----Building the model----
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 685)]             0         
                                                                 
 embedding_13 (Embedding)    (None, 685, 500)          25000500  
                                                                 
 batch_normalization_11 (Ba  (None, 685, 500)          2000      
 tchNormalization)                                               
                                                                 
 dropout_33 (Dropout)        (None, 685, 500)          0         
                                                                 
 conv1d_11 (Conv1D)          (None, 681, 32)           80032     
                                                                 
 dropout_34 (Dropout)        (None, 681, 32)           0         
                               

----Training the network----
Epoch 1/5


/Users/paperplanes/Library/Python/3.8/lib/python/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


347/347 - 193s - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0719 - 193s/epoch - 557ms/step
Epoch 2/5
347/347 - 420s - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0719 - 420s/epoch - 1s/step
Epoch 3/5
347/347 - 2038s - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0719 - 2038s/epoch - 6s/step
Epoch 4/5
347/347 - 2029s - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0719 - 2029s/epoch - 6s/step
Epoch 5/5
347/347 - 15470s - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0719 - 15470s/epoch - 45s/step
347/347 [==============================] - 7893s 23s/step - loss: 0.0000e+00 - accuracy: 0.0716
Train score: [0.0, 0.07159624248743057]
87/87 [==============================] - 2036s 24s/step - loss: 0.0000e+00 - accuracy: 0.0719
Validation score: [0.0, 0.07186710089445114]


In [65]:
MAX_SEQUENCE_LENGTH = 250
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=Xtrain.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(Xtrain,ytrain,epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5


/Users/paperplanes/Library/Python/3.8/lib/python/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


156/156 [==============================] - 197s 1s/step - loss: 0.0000e+00 - accuracy: 0.0714 - val_loss: 0.0000e+00 - val_accuracy: 0.0731
Epoch 2/5
156/156 [==============================] - 162s 1s/step - loss: 0.0000e+00 - accuracy: 0.0714 - val_loss: 0.0000e+00 - val_accuracy: 0.0731
Epoch 3/5
156/156 [==============================] - 161s 1s/step - loss: 0.0000e+00 - accuracy: 0.0714 - val_loss: 0.0000e+00 - val_accuracy: 0.0731
Epoch 4/5
156/156 [==============================] - 155s 993ms/step - loss: 0.0000e+00 - accuracy: 0.0714 - val_loss: 0.0000e+00 - val_accuracy: 0.0731


In [ ]:
result = model.evaluate(pad_test,ytest)